In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#https://huggingface.co/datasets/Ryan-sjtu/ffhq512-caption

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import os
import uuid
import numpy as np


#pretrained image to text 모델 불러오는 코드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

reward_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
reward_model.text_decoder.cls.predictions.decoder = nn.Linear(768, 1)
reward_model = reward_model.to("cuda")

In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    return total_params, trainable_params

count_parameters(model)
count_parameters(reward_model)

In [ ]:
print(model)

In [ ]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')


text = 'hihi my name is jimin and his name is chacha'
inputs = processor([raw_image], [text],padding=True, return_tensors="pt").to("cuda")
inputs['input_ids']

In [ ]:
#수집될 데이터 전처리 하는 코드
import re

def masking(text):
    parts = re.split(r'(\[.*?\]|\{.*?\})', text)  # 소괄호와 대괄호 둘 다 포함
    mask = []

    for part in parts:
        part = part.strip()
        if not part:
            continue  # 빈 문자열은 건너뛰기

        if part.startswith('{') and part.endswith('}'):
            mask_value = -1
            clean_text = part[1:-1]
        elif part.startswith('[') and part.endswith(']'):
            mask_value = 1
            clean_text = part[1:-1]
        else:
            mask_value = 0
            clean_text = part

        # 토큰 수 (BLIP processor 사용 시 [CLS], [SEP] 포함되므로 -2)
        part_tokens = processor(raw_image, clean_text, return_tensors="pt")['input_ids']
        mask.extend([mask_value] * (part_tokens.shape[1] - 2))

    # [CLS]와 [SEP] 토큰 고려해서 마스크 만들기 (ID 101, 102는 그냥 0으로 처리)
    final_mask = [0] + mask + [0]
    return final_mask

masking('hihi my {name} is jimin [and his] name is chacha')

In [ ]:
def simple_generate(model, processor, raw_image, text, max_length=60):
  text = "a photography of"
  inputs = processor(raw_image, text, return_tensors="pt").to("cuda")
  inputs['input_ids'] = inputs['input_ids'][:,:-1] #eos까지 만들어짐
  #inputs['pixel_values'] += torch.randn_like(inputs['pixel_values']) * 0.5

  encoder_outputs = model.vision_model(pixel_values=inputs['pixel_values'])

  max_length = 20


  for step in range(max_length):
      outputs = model.text_decoder(
          input_ids=inputs['input_ids'],
          encoder_hidden_states=encoder_outputs.last_hidden_state,
          encoder_attention_mask=torch.ones(encoder_outputs.last_hidden_state.shape[:-1], device="cuda"),
          return_dict=True)

      next_token_logits = outputs.logits[:, -1, :]

      next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

      inputs['input_ids'] = torch.cat([inputs['input_ids'], next_token], dim=-1)

      if next_token.item() == 102:
          break
  return inputs['input_ids']

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

output = simple_generate(model, processor, raw_image=raw_image, text="a photography of", max_length=60)
caption = processor.tokenizer.decode(output.squeeze(), skip_special_tokens=True)
caption

In [ ]:
!pip install datasets --quiet

In [ ]:
#FFHQ데이터를 허깅페이스 라이브러리에서 불러오기러오기
from datasets import load_dataset

ds = load_dataset("bitmind/ffhq-256")

In [ ]:
ds

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
class Custom_Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = np.array(self.data[idx])
        return x

In [ ]:
train_dataset = Custom_Dataset(ds['train']['image'])

In [ ]:
batch_size = 1
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#결과 저장 리스트
results = []

In [ ]:
#데이터 수집 코드
#수집할 데이터 형식: 선호하는/선호하지 않은 단어들 마킹, 전체 문장 점수 지정

ref_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
for epoch in range(1):
    for images in data_loader:
        image = images[0]
        print(image.shape)
        input_tensor = processor(image, return_tensors="pt").to("cuda")

        # 이미지 넘파이로 저장
        image_np = np.array(image)  # PIL 이미지를 넘파이 배열로 변환
        image_id = str(uuid.uuid4())  # 고유 ID 생성
        image_path = f"/content/drive/MyDrive/RLHF_data6/{image_id}.npy"  # .npy 형식으로 저장
        np.save(image_path, image_np)  # 넘파이로 저장

        plt.imshow(image)
        plt.axis('off')
        plt.show()

        # caption 생성
        output = model.generate(**input_tensor)
        print(processor.tokenizer.decode(output[0], skip_special_tokens=True))

        output = ref_model.generate(**input_tensor,
                                do_sample=True,
                                temperature=0.5,
                                top_k=20,
                                top_p=0.7,
                                repetition_penalty=1.2,
                                no_repeat_ngram_size=3,
                                max_new_tokens=60,
                                num_return_sequences=4)
        decoded_outputs = [processor.tokenizer.decode(o, skip_special_tokens=True) for o in output]

        for i, caption in enumerate(decoded_outputs):
            print(caption)
        print()
        for i, caption in enumerate(decoded_outputs):
            user_input = __builtins__.input()
            if user_input == 'p':
              break
            total_grade = __builtins__.input()
            results.append({
                "image_path": image_path,  # 넘파이 경로만 저장
                "edited_caption": user_input,
                "total_grade": total_grade
            })

In [ ]:
# 결과 저장 (JSONL)
output_path = "/content/drive/MyDrive/RLHF_data6/caption_feedback_text.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(len(results))

In [ ]:
def remove_brackets_only(text):
    # 대괄호([])와 중괄호({})만 제거하고 안의 내용은 유지
    text = text.replace('[', '').replace(']', '')
    text = text.replace('{', '').replace('}', '')
    return text

In [ ]:
import numpy as np
from PIL import Image

# RLHF_data4, RLHF_data5파일 위치로 수정해야 작동 됨
# 저장해 둔 데이터 불러오는 코드드
with open("/content/drive/MyDrive/RLHF_data4/caption_feedback_text.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]
with open("/content/drive/MyDrive/RLHF_data5/caption_feedback_text.jsonl", "r", encoding="utf-8") as f:
    data = data + [json.loads(line) for line in f]

with open("/content/drive/MyDrive/RLHF_data6/caption_feedback_text.jsonl", "r", encoding="utf-8") as f:
    data = data + [json.loads(line) for line in f]


from random import shuffle
print(len(data))

In [ ]:
reward_optimizer = optim.Adam(reward_model.parameters(), lr=1e-5)
reward_loss_fn = nn.MSELoss()
tanh = nn.Tanh()
batch_size = 4

In [ ]:
#reward_model학습
#선호하는 단어의 토큰은 1, 선호하지 않는 토큰은 0을 반환하도록 학습, 전체 문장에 대한 점수도 같이 반환하도록
batch_size = 4
for epoch in range(4):

  shuffle(data)
  for batch in range(len(data)//4):
      item = data[batch*4:(batch+1)*4]

      try:
        img_list = [np.load(item[i]["image_path"]) for i in range(4)]
      except:
        print('np ERROR')
        continue
      input_text = [remove_brackets_only(item[i]["edited_caption"]) for i in range(batch_size)]

      input_tensor = processor(img_list, input_text, padding=True, return_tensors="pt").to("cuda")
      output = reward_model.forward(**input_tensor)['logits'].squeeze()
      output[:,0] = 0
      target = torch.zeros_like(output)

      try:
        for i in range(batch_size):
          a = len(masking(item[i]["edited_caption"]))
          target[i,:a] = torch.tensor(masking(item[i]["edited_caption"]),dtype=torch.float)
          output[i,a:] = 0
        target[input_tensor['input_ids'] == 102] = torch.tensor([ (( int(item[j]["total_grade"]) - 1.0 )/2.0 - 1.0) for j in range(batch_size) ],dtype=torch.float).to("cuda")
      except:
        print('ERROR')
        print(item[i]["edited_caption"])
        continue

      if output.shape != target.shape:
        print(output.shape, target.shape)
        print('error')
        continue

      loss = reward_loss_fn(tanh(output), target)
      reward_optimizer.zero_grad()
      loss.backward()
      reward_optimizer.step()
      print(loss)


In [ ]:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
ref_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
optimizer = optim.Adam(model.parameters(), lr=1e-6)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#메모리 관리 코드
import gc
output = 0
loss = 0
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#FFHQ데이터셋에 대해서 생성된 여러 text들의 점수를 각각 부여하고, 가장 높은 점수를 받은 데이터들을 모아서
#supervised learning을 진행하기
batch_size = 8
preference_mean = 0
c = 0

for epoch in range(1):
    for images in data_loader:
        image = images[0]
        #plt.imshow(image)#
        #plt.show()#
        input_tensor = processor(image, return_tensors="pt").to("cuda")
        target = ref_model.generate(**input_tensor,
                                do_sample=True,
                                temperature=0.5,
                                top_k=20,
                                top_p=0.8,
                                repetition_penalty=1.2,
                                no_repeat_ngram_size=3,
                                max_new_tokens=60,
                                num_return_sequences=batch_size)[:,1:]

        decoded_outputs = [processor.tokenizer.decode(o, skip_special_tokens=True) for o in target]

        target = torch.concat((torch.tensor([[101]]*batch_size,dtype=torch.int).to(model.device),target),dim=1)
        input_tensor = processor([image]*batch_size, return_tensors="pt").to("cuda")
        input_tensor['input_ids'] = target
        output = model.forward(**input_tensor)['logits']

        output[target == 0] = 0
        output[target == 102] = 0

        mask = (target == 0)
        mask2 = (target == 102)
        mask3 = (target != 0) & (target != 101) & (target != 102)

        target = F.one_hot(target, num_classes=30524).to(torch.float)
        with torch.no_grad():
          preference = tanh(reward_model(**input_tensor)['logits'].squeeze())
          preference[mask] = 0
          preference[:,0] = 0
          c += 1
          preference_mean = (preference_mean*(c-1)+preference[mask3].mean())/c
          preference[mask2] *= 16
          #preference[mask3] -= preference_mean

        preference = preference.sum(dim=1)

        max = -100000
        where = 0
        for i in range(output.shape[0]):
          #print(decoded_outputs[i])
          #print(preference[i].detach().item())
          if max < preference[i].detach().item():
            max = preference[i].detach().item()
            where = i
        if max > 0:
          loss = loss_fn(output[[where],:-1],target[[where],1:].detach())
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          print(loss)

In [ ]:
#랜덤한 20개의 이미지에 대해서 pretrained 모델, 우리 모델간의 output비교
for epoch in range(20):
    for images in data_loader:
        image = images[0]
        plt.imshow(image)
        plt.show()
        input_tensor = processor(image, return_tensors="pt").to("cuda")

        ref_model_output = ref_model.generate(**input_tensor)
        print(processor.tokenizer.decode(ref_model_output[0], skip_special_tokens=True))
        model_output = model.generate(**input_tensor)
        print(processor.tokenizer.decode(model_output[0], skip_special_tokens=True))
        print()
        break